In [1]:
from utils.data.create_local_t5data import get_trial_data
from datasets import get_testing_data
from utils_f import get_config
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from datetime import datetime
from utils.plot.plot_true_vs_pred_mvmnt import plot_true_vs_pred_mvmnt
import torch
from utils_f import get_config_from_file, set_seeds, set_device
from datasets import get_trial_data, chop, smooth_spikes
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import plotly.graph_objects as go
from matplotlib import cm, colors
import matplotlib.pyplot as plt
import shutil
import os
import os.path as osp
import sys
import wandb
import pandas as pd
import copy
import pickle

/home/dmifsud/miniconda3/envs/ndtu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# path = '/home/dmifsud/Projects/NDT-U/runs/train/honest-sweep-10/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/stellar-sweep-1/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/sweet-serenity-164/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/polar-violet-159/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/neat-sweep-6/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/resilient-sweep-7/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/bumbling-sweep-28/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/clean-energy-185/last.pt'
# path = '/home/dmifsud/Projects/NDT-U/runs/train/zesty-sweep-17/last.pt'
path = '/home/dmifsud/Projects/NDT-U/runs/train/misunderstood-elevator-1/last.pt'

In [3]:
name = path[:path.rindex('/')].split('/')[-1]
config = get_config_from_file(path[:path.rindex('/')+1]+'config.yaml')
if not os.path.isdir(f"plots/{name}"): os.makedirs(f"plots/{name}")
shutil.copyfile(path[:path.rindex('/')+1]+'config.yaml', f"plots/{name}/config.yaml")

'plots/misunderstood-elevator-1/config.yaml'

In [4]:
# datasets = get_testing_data(config)

In [5]:
# set_seeds(config)
# set_device(config, {})
# device = torch.device('cuda:0')

# model = torch.load(path).to(device)
# model.name = name

# model.eval()

# session_csv = pd.read_csv(f'{config.data.dir}/sessions.csv')

In [6]:
# dataset = copy.deepcopy(datasets[session]) # do not want to run xcorr on test data

In [7]:
# def run(dataset, config, model, session, device, cond_list):
#     if osp.exists('session_trials/cond_list.pickle'):
#         with open('session_trials/cond_list.pickle', 'rb') as handle:
#             cond_list = pickle.load(handle)
#     else:
#         cond_list = None
#     session_csv = pd.read_csv(f'{config.data.dir}/sessions.csv')

#     if config.data.rem_xcorr: 
#         corr, corr_chans = dataset.get_pair_xcorr('spikes', threshold=0.2, zero_chans=True)
        
#     dataset.resample(config.data.bin_size / 1000)
#     dataset.smooth_spk(60, name='smth') # for use if we want to take mean and std of smth values

#     failed_trials = ~dataset.trial_info['is_successful'] 
#     center_trials = dataset.trial_info['is_center_target']
#     ol_block = session_csv.loc[session_csv['session_id'] == session, 'ol_blocks'].item()
#     cl_blocks =  ~dataset.trial_info['block_num'].isin([ol_block]).values.squeeze()

#     spks = dataset.data[dataset.data['blockNums'].isin([ol_block]).values.squeeze()].spikes.to_numpy()
#     spks_idx = dataset.data[dataset.data['blockNums'].isin([ol_block]).values.squeeze()].spikes.index

#     n_channels = dataset.data.spikes.shape[-1]

#     n_heldout = int(config.data.heldout_pct * n_channels)
#     n_heldin = n_channels - n_heldout
#     np.random.seed(config.setup.seed)
#     heldout_channels = np.random.choice(n_channels, n_heldout, replace=False)
#     heldin_channels = torch.ones(n_channels, dtype=bool)
#     heldin_channels[heldout_channels] = False

#     chopped_spks = chop(np.array(spks[:, heldin_channels]), 30, 29)
#     hi_chopped_spks = torch.Tensor(chopped_spks).to(device)


#     names = [session for i in range(hi_chopped_spks.shape[0])]
#     with torch.no_grad():
#         _, output = model(hi_chopped_spks, names)


#     # factors_df = pd.DataFrame(outputs[:, -1, :], index=spks_idx[29:], columns=pd.MultiIndex.from_tuples([('factors', f'{i}') for i in range(output.shape[-1])]))
#     factors_df = pd.DataFrame(output[:, -1, :].cpu().numpy(), index=spks_idx[29:], columns=pd.MultiIndex.from_tuples([('factors', f'{i}') for i in range(output.shape[-1])]))
#     dataset.data = pd.concat([dataset.data, factors_df], axis=1)

#     dataset.smooth_spk(config['data']['smth_std'], signal_type='factors', name='smth')

#     ignored_trials = failed_trials | center_trials | cl_blocks
#     ignored_trials[1] = True

#     trial_data = dataset.make_trial_data(
#         align_field='start_time',
#         align_range=(0, config.data.trial_len),
#         allow_overlap=True,
#         ignored_trials= ignored_trials
#     )

#     trial_data.sort_index(axis=1, inplace=True)
#     trial_data['X&Y'] = list(zip(trial_data['targetPos']['x'], trial_data['targetPos']['y']))
#     trial_data['condition'] = 0

#     if cond_list == None:
#         cond_list = list(zip(trial_data['X&Y'].unique(), np.arange(1,9)))
#     for xy, id in cond_list:
#         indices = trial_data.index[trial_data['X&Y'] == xy]
#         trial_data.loc[indices, 'condition'] = id
#         print(id, xy)

#     factors = []
#     for cond_id, trials in trial_data.groupby('condition'):
#         for trial_id, trial in trials.groupby('trial_id'):
#             factors.append(trial.factors_smth.to_numpy())

#     np.save(f'session_factors/{session}.npy', np.array(factors))

#     with open(f'session_trials/{session}.pickle', 'wb') as handle:
#         pickle.dump(trial_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     if not osp.exists('session_trials/cond_list.pickle'):  
#         with open(f'session_trials/cond_list.pickle', 'wb') as handle:
#             pickle.dump(cond_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
session_list = [
   't5.2021.05.05',
   't5.2021.05.17',
   't5.2021.05.19',
   't5.2021.05.24',
   # 't5.2021.05.26',
   't5.2021.06.02',
   't5.2021.06.04',
   't5.2021.06.07',
   't5.2021.06.23',
   't5.2021.06.28',
   't5.2021.06.30',
   't5.2021.07.07',
   't5.2021.07.08',
   't5.2021.07.12',
   't5.2021.07.14',
   't5.2021.07.19',
   't5.2021.07.21',   
]   
factors_list = [
   'session_factors/t5.2021.05.05_.npy',
   'session_factors/t5.2021.05.17_.npy',
   'session_factors/t5.2021.05.19_.npy',
   'session_factors/t5.2021.05.24_.npy',
   # 'session_factors/t5.2021.05.26_.npy',
   'session_factors/t5.2021.06.02_.npy',
   'session_factors/t5.2021.06.04_.npy',
   'session_factors/t5.2021.06.07_.npy',
   'session_factors/t5.2021.06.23_.npy',
   'session_factors/t5.2021.06.28_.npy',
   'session_factors/t5.2021.06.30_.npy',
   'session_factors/t5.2021.07.07_.npy',
   'session_factors/t5.2021.07.08_.npy',
   'session_factors/t5.2021.07.12_.npy',
   'session_factors/t5.2021.07.14_.npy',
   'session_factors/t5.2021.07.19_.npy',
   'session_factors/t5.2021.07.21_.npy',   
]   
trials_list = [
   'session_trials/t5.2021.05.05_.pickle',
   'session_trials/t5.2021.05.17_.pickle',
   'session_trials/t5.2021.05.19_.pickle',
   'session_trials/t5.2021.05.24_.pickle',
   # 'session_trials/t5.2021.05.26_.pickle',
   'session_trials/t5.2021.06.02_.pickle',
   'session_trials/t5.2021.06.04_.pickle',
   'session_trials/t5.2021.06.07_.pickle',
   'session_trials/t5.2021.06.23_.pickle',
   'session_trials/t5.2021.06.28_.pickle',
   'session_trials/t5.2021.06.30_.pickle',
   'session_trials/t5.2021.07.07_.pickle',
   'session_trials/t5.2021.07.08_.pickle',
   'session_trials/t5.2021.07.12_.pickle',
   'session_trials/t5.2021.07.14_.pickle',
   'session_trials/t5.2021.07.19_.pickle',
   'session_trials/t5.2021.07.21_.pickle',   
]   

In [9]:
# for i in session_list:
#     !python run_model.py $i $path

In [10]:
# factors = []
# for session in config.data.pretrain_sessions:    
#     for cond_id, trials in trialized_data[session].groupby('condition'):
#         for trial_id, trial in trials.groupby('trial_id'):
#             factors.append(trial.factors_smth.to_numpy())


In [11]:

factors = []
for i in factors_list:    
    factors.append(np.load(i))

factors = np.concatenate(factors, 0)
fs = factors.shape
print(fs)
# factors = factors.reshape((fs[0] * fs[1], fs[2]))
pca = PCA(n_components=7)
# pca = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=7))])
pca.fit(factors)

(147400, 128)


PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [12]:
trialized_data = {}
for sess, trials in zip(session_list, trials_list):
    with open(trials, 'rb') as handle:
        trialized_data[sess] = pickle.load(handle)

In [13]:
trialized_data.keys()

dict_keys(['t5.2021.05.05', 't5.2021.05.17', 't5.2021.05.19', 't5.2021.05.24', 't5.2021.06.02', 't5.2021.06.04', 't5.2021.06.07', 't5.2021.06.23', 't5.2021.06.28', 't5.2021.06.30', 't5.2021.07.07', 't5.2021.07.08', 't5.2021.07.12', 't5.2021.07.14', 't5.2021.07.19', 't5.2021.07.21'])

In [14]:
wandb.init(project='High Dropout Sweep', name='all_sessions_aligned')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: emory-bg2 (use `wandb login --relogin` to force relogin)


In [15]:
# COND AVG
    
fig = go.Figure()
for session in session_list:    
    for cond_id, trials in trialized_data[session].groupby('condition'):
        avg_trials = []
        for trial_id, trial in trials.groupby('trial_id'):
            avg_trials.append(pca.transform(trial.factors_smth))
        avg_trials = np.array(avg_trials).mean(0)
        fig.add_trace(
            go.Scatter3d(
                x=avg_trials[:, 0], 
                y=avg_trials[:, 1], 
                z=avg_trials[:, 2],
                mode='lines',
                line=dict(color=f'{colors.rgb2hex(cm.tab10(cond_id))}'),
            )
        )

fig.update_layout(
    width=430,
    height=410,
    autosize=False,
    showlegend=False,
    title={
        'text': "Condition Averaged PCs",
        'y':0.96,
        'yanchor': 'bottom'
    },
    scene=dict(
        xaxis_showspikes=False,
        yaxis_showspikes=False,
        zaxis_showspikes=False,
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3",
        camera=dict(
            center=dict(
                x=0.065,
                y=0.0,
                z=-0.075,
                # z=-0.12,
            ),
            eye=dict(
                x=1.3, 
                y=1.3, 
                z=1.3
            )
        ),
        aspectratio = dict( x=1, y=1, z=1 ),
        aspectmode = 'manual'
    ),
)

fig.update_layout(margin=dict(r=0, l=0, b=0, t=20))
config2 = {'displayModeBar': False}
html_string = fig.to_html(config=config2)

wandb.log({'PC_plots_cond_avg': wandb.Html(html_string, inject=False)})

In [16]:
# SINGLE TRIAL

fig = go.Figure()
for session in session_list:    
    for cond_id, trials in trialized_data[session].groupby('condition'):
        for trial_id, trial in trials.groupby('trial_id'):
            pc_factors = pca.transform(trial.factors_smth)
            fig.add_trace(
                go.Scatter3d(
                    x=pc_factors[:, 0], 
                    y=pc_factors[:, 1], 
                    z=pc_factors[:, 2],
                    mode='lines',
                    line=dict(color=f'{colors.rgb2hex(cm.tab10(cond_id))}'),
                )
            )

fig.update_layout(
    width=430,
    height=410,
    autosize=False,
    showlegend=False,
    title={
        'text': "Single Trial PCs",
        'y':0.96,
        'yanchor': 'bottom'
    },
    scene=dict(
        xaxis_showspikes=False,
        yaxis_showspikes=False,
        zaxis_showspikes=False,
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3",
        camera=dict(
            center=dict(
                x=0.065,
                y=0.0,
                z=-0.075,
                # z=-0.12,
            ),
            eye=dict(
                x=1.3, 
                y=1.3, 
                z=1.3
            )
        ),
        aspectratio = dict( x=1, y=1, z=1 ),
        aspectmode = 'manual'
    ),
)

fig.update_layout(margin=dict(r=0, l=0, b=0, t=20))
config2 = {'displayModeBar': False}
html_string = fig.to_html(config=config2)

wandb.log({'PC_plots_single_trials': wandb.Html(html_string, inject=False)})

In [17]:
wandb.finish()